<a href="https://colab.research.google.com/github/elio-cs/Pytorch/blob/main/SCAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Apr 25 08:54:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocess

In [ ]:
"""

|- root_dir               -> FODB/
   |- dir                 -> FODB/D01_Motorola_E3_1/
      |- sub_dir          -> FODB/D01_Motorola_E3_1/orig/
         |- image         -> FODB/D01_Motorola_E3_1/orig/D01_img_orig_0001.jpg
         |- ...
      |- ...
   |- ...

"""

'\n\n|- root_dir               -> FODB/\n   |- dir                 -> FODB/D01_Motorola_E3_1/\n      |- sub_dir          -> FODB/D01_Motorola_E3_1/orig/\n         |- image         -> FODB/D01_Motorola_E3_1/orig/D01_img_orig_0001.jpg\n         |- ...\n      |- ...\n   |- ...\n\n'

In [ ]:
# mkdir train_set and test_set

import os
import cv2


photo_type = "orig"

train_path = "/content/drive/MyDrive/Dataset/FODB/Train/orig_64"
test_path = "/content/drive/MyDrive/Dataset/FODB/Test/orig_64"

root_path = "/content/drive/MyDrive/Dataset/FODB"
root_dir = os.walk(root_path)

for dir_path_1, dir_list_1, file_list_1 in root_dir:
  for dir_1 in dir_list_1:
    dir_path = os.path.join(dir_path_1, dir_1)
    dir = os.walk(dir_path)
    for dir_path_2, dir_list_2, file_list_2 in dir:
      for dir_2 in dir_list_2:
        if dir_2 == "orig":
          sub_dir_path = os.path.join(dir_path_2, dir_2)
          sub_dir = os.walk(sub_dir_path)
          for dir_path_3, dir_list_3, image_list in sub_dir:
           print(dir_1)
           print("train:")
           for image in image_list[0:100]:
              image_path = os.path.join(sub_dir_path, image)
              img = cv2.imread(image_path)
              middle_point = [int(img.shape[0]/2), int(img.shape[1]/2)]
              cropped = img[middle_point[0] - 32 : middle_point[0] + 32, middle_point[1] - 32 : middle_point[1] + 32]
              save_path = os.path.join(train_path, image)
              print(save_path)
              cv2.imwrite(save_path, cropped)
           print("test:")
           for image in image_list[100:]:
              image_path = os.path.join(sub_dir_path, image)
              img = cv2.imread(image_path)
              middle_point = [int(img.shape[0]/2), int(img.shape[1]/2)]
              cropped = img[middle_point[0] - 32 : middle_point[0] + 32, middle_point[1] - 32 : middle_point[1] + 32]
              print(save_path)
              save_path = os.path.join(test_path, image)
              cv2.imwrite(save_path, cropped)

# Make Label

In [ ]:
train_path = "/content/drive/MyDrive/Dataset/FODB/Train/orig_64"
train_dir = os.walk(train_path)

for dir_path, dir_list, image_list in train_dir:
  print(len(image_list))
  

2700


# Model

In [ ]:
# utils/mypath.py
import os

In [ ]:
class Dataset_path(object):
  def db_root_dir(database=''):
    db_names = {'FODB'}
    assert(database in db_names)

    if database == 'FODB':
      return "/content/drive/MyDrive/Dataset/FODB"

In [ ]:
# dataset.py
import os
import pickle
import sys
import numpy as np
import torch
from PIL import Image
from torch.utils.data import Dataset
from torchvision.datasets.utils import check_integrity, download_and_extract_archive

In [ ]:
class FODB(Dataset):
  """`FODB<https://faui1-files.cs.fau.de/public/mmsec/datasets/fodb/>`
    Args:
      root:(string): Root directory of dataset where directory
      train (bool, optional): If True, creates dataset from training set, otherwise
            creates from test set.
      transform (callable, optional): A function/transform that takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
  """
  def __init__(self, root = Dataset_path.db_root_dir('FODB'), train=True, transform=None):
    super(FODB,self).__init__()
    self.root = root
    self.transform = transform
    self.train = train
    self.classes = ['D01_Motorola_E3_1', 'D02_LG_OptimusL50_1', 'D03_Wiko_Lenny2_1', 'D04_LG_G3_1', 
                    'D05_Apple_iPhone6s_1', 'D06_LG_G6_1', 'D07_Motorola_Z2Play_1', 'D08_Motorola_G8Plus_1', 
                    'D09_Samsung_GalaxyS4mini_1', 'D10_Samsung_GalaxyJ1_1', 'D11_Samsung_GalaxyJ3_1', 'D12_Samsung_GalaxyStar5280_1', 
                    'D13_Sony_XperiaE5_1', 'D14_Apple_iPhone3_1', 'D15_Samsung_GalaxyA6_1', 'D16_Samsung_GalaxyA6_2', 
                    'D17_Apple_iPhone7_1', 'D18_Samsung_GalaxyS4_1', 'D19_Apple_iPhone8Plus_1', 'D20_Google_Pixel3_1',
                    'D21_Google_Nexus5_1', 'D22_BQ_AquarisX_1', 'D23_Huawei_P9lite_1', 'D24_Huawei_P8lite_1', 
                    'D25_Huawei_P9lite_2', 'D26_Huawei_P20lite_1', 'D27_Google_PixelXL_1']
    if self.train:
      data_list = self.train_list
    else:
      data_list = self.test_list
  
    self.data = []
    self.target = []
    for file_name, checksum in data_list:
      file_path = os.path.join(self.root, self.base_folder, file_name)
      with open(file_path, 'rb') as f:
        if sys.version_info[0] == 2:
          entry = pickle.load(f)
        
